# Musical Viterbi Test

Define a baby HMM melody-harmony model. The hidden states are five diatonic chords in C major, the observation states are the C major pentatonic scale.

The complication with this model is that the viterbi algorithm accepts a vector of observation states (this could record the relative importances of the notes in a bar).

The viterbi algorithm uses the `emission_dist_prob` function to measure the probability of an observation being emitted by a given state. This function simply takes the dot product of the emission distribution and the observation vector.

Short answer: It works!


### Set up model

- Model needs start probabilities (pi), transition matrix (A), and emission distributions (B).
- Emission probs have type `np.array`. Other quantities should really be `np.array`s too?
- state names and observation state names are not used, but could be useful if the transition matrix was a `np.array` and not a `dict`.

In [1]:
import numpy as np

# Model:
states = ("C", "Dmin", "F", "G", "Amin") # possible states
obs_states = ('C', 'D', 'E', 'G', 'A')


start_probability = {"C": 0.5, "Dmin": 0.3, "F": 0.1, "G": 0.05, "Amin": 0.05} # pi

transition_probability = { # State to State transition matrix: A
    "C" : {'C': 0.1, 'Dmin': 0.1, 'F': 0.3, 'G': 0.1, 'Amin': 0.4}, 
    "Dmin" : {'C': 0.1, 'Dmin': 0.1, 'F': 0.2, 'G': 0.5, 'Amin': 0.1},  
    "F" : {'C': 0.2, 'Dmin': 0.2, 'F': 0.2, 'G': 0.2, 'Amin': 0.2},   
    "G" : {'C': 0.5, 'Dmin': 0.05, 'F': 0.1, 'G': 0.3, 'Amin': 0.05}, 
    "Amin" : {'C': 0.3, 'Dmin': 0.3, 'F': 0.3, 'G': 0.05, 'Amin': 0.05}, 
}

# Use np.arrays for emission probs.

emission_probability = { # State to Observation matrix: B
    "C" : np.array([0.3, 0.05, 0.3, 0.3, 0.05]), 
    "Dmin" : np.array([0.1, 0.3, 0.1, 0.2, 0.3]), 
    "F" : np.array([0.2, 0.05, 0.3, 0.05, 0.4]),  
    "G" : np.array([0.1, 0.3, 0.1, 0.4, 0.1]), 
    "Amin" : np.array([0.3, 0.05, 0.3, 0.05, 0.3]), 
}

### Viterbi Algorithm

- Got this working with only minor adjustments. 
- Code is somewhat labelled now.
- Thanks Henrik for guiding me towards understanding! :-)
- Not quite happy with this implementation. For instance, why not store the max_prob element at each step during the forward pass? It then has to backtrack from the end of V to find (again) all the `max_probs`.

In [2]:
# The viterbi algorithm: (thanks wikipedia)

def emission_dist_prob(emission_distribution, observation):
    # Probability measure of observation dist in emission_distritbution
    return np.dot(emission_distribution, observation)

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] # Probability Table Prepared
    
    # Generate Probs for 0th hidden state.
    for st in states:
        V[0][st] = {"prob": start_p[st] * emission_dist_prob(emit_p[st],obs[0]), "prev": None}
        
    # Generate Probs for 1st-nth hidden states 
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"] * trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emission_dist_prob(emit_p[st],obs[t]) # mult by emission prob.
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    
    # Prepare to identify best hidden state sequence from V
    output = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    
    # Get most probable final state from the end of V
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            output.append(st)
            previous = st
            break
            
    # Get the most probable state in each previous step of the list.
    for t in range(len(V) - 2, -1, -1):
        output.insert(0, V[t + 1][previous]["prev"]) # put the 'prev' state at index 0 in output
        previous = V[t + 1][previous]["prev"] # store previous 'prev' state.
    
    # Return the list of states
    return(output, max_prob, V)

### Predict Chords for a simple melody

Use the Viterbi algorithm to predict chords for a melody with one note in each bar.

In [3]:
# some note vectors equivalent to ['C', 'C', 'G', 'G', 'A', 'A', 'G']
observations = [   
    np.array([1., 0., 0., 0., 0.]),
    np.array([1., 0., 0., 0., 0.]),
    np.array([0., 0., 0., 1., 0.]),
    np.array([0., 0., 0., 1., 0.]),
    np.array([0., 0., 0., 0., 1.]),
    np.array([0., 0., 0., 0., 1.]),
    np.array([0., 0., 0., 1., 0.]),
]

pred, prob, V = viterbi(observations, states, start_probability, transition_probability, emission_probability)

print("Predicted path:", pred)
print("Likelihood:", prob)

Predicted path: ['C', 'F', 'G', 'C', 'Amin', 'Dmin', 'G']
Likelihood: 2.3328e-07
